In [6]:
# XGBoost 실습
import pandas as pd

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train.drop('index', axis = 1, inplace = True)
test.drop('index', axis = 1, inplace = True)

In [7]:
train

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,white
1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,red
2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,white
3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,white
4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5492,5,7.7,0.150,0.29,1.3,0.029,10.0,64.0,0.99320,3.35,0.39,10.1,white
5493,6,6.3,0.180,0.36,1.2,0.034,26.0,111.0,0.99074,3.16,0.51,11.0,white
5494,7,7.8,0.150,0.34,1.1,0.035,31.0,93.0,0.99096,3.07,0.72,11.3,white
5495,5,6.6,0.410,0.31,1.6,0.042,18.0,101.0,0.99195,3.13,0.41,10.5,white


In [8]:
train_one = pd.get_dummies(train)
test_one = pd.get_dummies(test)

In [9]:
model = XGBClassifier()

In [10]:
X = train_one.drop('quality', axis = 1)
y = train_one['quality']

model.fit(X, y)

C:\Users\freshman\Desktop\AI\ai\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:20:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [11]:
pred = model.predict(test_one)

In [12]:
submission = pd.read_csv('data/sample_submission.csv')
submission['quality'] = pred
submission.to_csv('xgb_pred.csv', index = False)

In [18]:
# LightGBM 실습
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train.drop('index', axis = 1, inplace = True)
test.drop('index', axis = 1, inplace = True)

In [19]:
train_one = pd.get_dummies(train)
test_one = pd.get_dummies(test)

In [20]:
from lightgbm import LGBMClassifier

model = LGBMClassifier()

In [21]:
X = train_one.drop('quality', axis = 1)
y = train_one['quality']

In [22]:
model.fit(X, y)

LGBMClassifier()

In [23]:
pred = model.predict(test_one)

In [24]:
submission = pd.read_csv('data/sample_submission.csv')
submission['quality'] = pred
submission.to_csv('lgbm_pred.csv', index = False)

In [25]:
# Stratified k-fold로 교차검증 후, LightGBM을 이용해 와인 품질 분류
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

skf = StratifiedKFold(n_splits = 5)

In [26]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train.drop('index', axis = 1, inplace = True)
test.drop('index', axis = 1, inplace = True)

X = train_one.drop('quality', axis = 1)
y = train_one['quality']

In [27]:
cnt = 1
acc = 0
for train_idx, valid_idx in skf.split(X, y):
    train_data = train_one.iloc[train_idx]
    valid_data = train_one.iloc[valid_idx]
    
    model = LGBMClassifier()
    
    train_X = train_data.drop('quality', axis = 1)
    train_y = train_data['quality']
    
    model.fit(train_X, train_y)
    
    valid_X = valid_data.drop('quality', axis = 1)
    valid_y = valid_data['quality']
    
    pred = model.predict(valid_X)
    
    print(f"{cnt}번째 모델 정확도 :", accuracy_score(pred, valid_y))
    acc += accuracy_score(pred, valid_y)
    cnt += 1
    
print('모델 정확도 평균 :', acc / 5)

1번째 모델 정확도 : 0.6609090909090909
2번째 모델 정확도 : 0.6581818181818182
3번째 모델 정확도 : 0.6606005459508644
4번째 모델 정확도 : 0.632393084622384
5번째 모델 정확도 : 0.6469517743403094
모델 정확도 평균 : 0.6518072628008933


In [29]:
# Voting Classifier 실습
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train.drop('index', axis = 1, inplace = True)
test.drop('index', axis = 1, inplace = True)

X = train_one.drop('quality', axis = 1)
y = train_one['quality']

LGBM = LGBMClassifier()
XGB = XGBClassifier()
RF = RandomForestClassifier()

VC = VotingClassifier(estimators = [('rf', RF), ('xgb', XGB), ('lgbm', LGBM)], voting = 'soft')

X = train_one.drop('quality', axis = 1)
y = train_one['quality']

VC.fit(X, y)

pred = VC.predict(test_one)

C:\Users\freshman\Desktop\AI\ai\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:46:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [30]:
pred

array([5, 6, 6, 5, 6, 6, 6, 6, 6, 7, 6, 6, 7, 6, 6, 5, 6, 6, 6, 6, 6, 6,
       5, 7, 6, 5, 5, 5, 5, 5, 7, 6, 5, 6, 5, 6, 5, 5, 6, 6, 6, 5, 6, 6,
       6, 7, 5, 6, 4, 6, 5, 5, 6, 4, 5, 6, 6, 5, 5, 6, 5, 6, 6, 4, 6, 6,
       5, 5, 6, 5, 5, 5, 5, 6, 6, 5, 6, 6, 6, 7, 6, 6, 5, 5, 6, 5, 5, 5,
       6, 7, 6, 6, 6, 6, 6, 7, 6, 6, 6, 7, 5, 6, 6, 5, 5, 6, 6, 6, 7, 5,
       6, 6, 6, 5, 5, 6, 6, 7, 7, 6, 6, 5, 6, 6, 5, 5, 6, 6, 6, 7, 5, 5,
       6, 5, 5, 6, 6, 7, 5, 5, 5, 6, 5, 5, 5, 5, 7, 6, 7, 6, 5, 8, 6, 5,
       6, 5, 5, 5, 6, 5, 5, 6, 6, 6, 6, 5, 6, 6, 5, 6, 5, 5, 6, 5, 6, 6,
       6, 5, 6, 6, 6, 6, 7, 7, 5, 6, 6, 6, 5, 6, 6, 6, 5, 6, 6, 6, 7, 5,
       5, 5, 6, 5, 6, 5, 5, 5, 6, 6, 6, 6, 7, 5, 5, 6, 6, 6, 6, 6, 6, 7,
       6, 5, 6, 6, 7, 6, 6, 5, 6, 5, 6, 7, 7, 6, 6, 6, 6, 6, 6, 7, 6, 6,
       6, 6, 7, 6, 6, 7, 6, 6, 5, 6, 7, 7, 6, 6, 6, 7, 7, 6, 7, 5, 5, 7,
       5, 5, 6, 5, 6, 6, 6, 5, 5, 5, 6, 5, 7, 7, 5, 6, 6, 6, 6, 6, 7, 7,
       6, 6, 6, 6, 5, 5, 5, 6, 5, 7, 8, 5, 5, 7, 7,

In [31]:
submission = pd.read_csv('data/sample_submission.csv')
submission['quality'] = pred
submission.to_csv('VC.csv', index = False)